# Phase 2 Quantum Algorithms

## Pavan Kumar, Kitty Yeung

In phase 1, we learned about the components of quantum computing – qubits, gates and
measurements, which can be put together to form algorithms. An algorithm takes in information from
qubit inputs, conducts a manipulation on the qubits, and then outputs desired results. We will be learning
about a few algorithms that were developed as foundations to quantum computing logics in a historical
order: Deutsch's algorithm, Deutsch-Jozsa algorithm, Grover's algorithm, Simon's algorithm and Shor's
algorithm. These algorithms solve certain problems more efficiently than classical algorithms. (There are
many other quantum algorithms. For the interested and curious readers, take a look at
[http://quantumalgorithmzoo.org/](http://quantumalgorithmzoo.org/) ) Before going deeper into the algorithms, we need to start with a
concept called "Quantum Oracle" which is an operation that is used as an input to another algorithm. In
each session, we will practice our understanding with a Q# exercise.

## 2.1 Quantum Oracles PHASE 2 Quantum Algorithms

* [Q# exercise: Oracles](./2-Quantum_Algorithms/1-Quantum_Oracles.ipynb#qexercise)

An oracle can be thought of as a black box that takes some inputs and gives some outputs. It can be
represented using a circuit diagram below.

![Figure 2.1.1](img/Figure211.png)

_Figure 2.1.1 A generic oracle represented by a circuit diagram._

As an example, this diagram can be an oracle that takes a three-bit integer and returns the reminder when
it is divided by 4 with these possible outputs:

<table>
  <tr>
    <th>x &nbsp; &nbsp; &nbsp; </th>
    <th>y = f(x) = x % 4 &nbsp; &nbsp; </th>
  </tr>
  <tr>
    <td> 0 </td>
    <td> 0 </td>
  </tr>
  <tr>
    <td> 1 </td>
    <td> 1 </td>
  </tr>
  <tr>
    <td> 2 </td>
    <td> 2 </td>
  </tr>
  <tr>
    <td> 3 </td>
    <td> 3 </td>
  </tr>
  <tr>
    <td> 4 </td>
    <td> 0 </td>
  </tr>
  <tr>
    <td> 5 </td>
    <td> 1 </td>
  </tr>
  <tr>
    <td> 6 </td>
    <td> 2 </td>
  </tr>
  <tr>
    <td> 7 </td>
    <td> 3 </td>
  </tr>
</table>

_Table 2.1.1 Input and output table for a particular oracle as an example._

We could build such a circuit using classical computing gates. But there are quantum algorithms
proven to be more efficient than classical ones. We need a quantum oracle that can do the above function
to build quantum algorithms, with a limitation that all the quantum gates need to be unitary (session 1.2).

Let's say that the initial number is encoded in a 3-qubit quantum state, x. And the function f(x)
is implemented as a unitary quantum gate A so that

y = f(x) = Ax.

Since A is unitary, A$^{−1}$ exists and it should be the same as A${^\dagger}$. This implies

A${^\dagger}$(y) = A$^{−1}$(y) = A$^{−1}$(Ax) = x.

This means that given any output, applying A${^\dagger}$ should revert to the corresponding input. However, as seen
in Table 2.1.1, both f(3) and f(7) equal to 3. Applying a unitary gate A${^\dagger}$ on f(3) and f(7) would not
return distinguishable results. Our assumption that such a matrix A can exist is wrong. This is a common
problem for designing quantum oracles. Yet, there is a work-around if we use some extra qubits.

For the same function, consider the following quantum circuit:

![Figure 2.1.2](img/Figure212.png)

_Figure 2.1.2 Quantum Oracle._

Inputs x and y can contain multiple numbers of qubits. As one can see, this circuit leaves the input x intact,
but it does an XOR on y with f(x) (in this example y = f(x) = x % 4). Here we are assuming that x and y are
encoded using three qubits each. The size of the input state will therefore be 64 ( $2^6$ ). The dimension of
the unitary matrix U will be 64 x 64. Don't worry about the exact contents of U right now. We are
assuming that such a unitary matrix exists and test if we arrive at any contradictions.

Let's evaluate this circuit for x= 3 = |011⟩, with y initialized to 0 = |000⟩ and refer to Table 2.1.1 for results of f(x):

U( |011⟩, |000⟩ ) = ( |011⟩, |000⟩ $\oplus$ F( |011⟩ ) ) = ( |011⟩, |000⟩ $\oplus$ |011⟩ ) = ( |011⟩, |011⟩ ),

It has the following circuit representation:

![Figure oracle 011](img/oracle011.png)

Since we have initialized y to 0, the output we need will be in the bottom 3 qubits (because y $\oplus$ f(x) = f(x), if y = 0). 
Take the outputs of the above circuit and apply the same circuit again on them:

U( |011⟩, |011⟩ ) = ( |011⟩, |011⟩ $\oplus$ f( |011⟩ ) ) = ( |011⟩, |011⟩ $\oplus$ |011⟩ ) = ( |011⟩, |000⟩ ).

It has a circuit representation:

![Figure oracle 011 again](img/oracle011again.png)

This demonstrates that applying the same gate U on the outputs of the first circuit recovers the original inputs.

Now, let's evaluate the quantum oracle for x = 7 = |111⟩, with y initialized to 0 = |000⟩ :

U( |111⟩, |000⟩ ) = ( |111⟩, |000⟩ $\oplus$ f( |111⟩ ) ) = ( |111⟩, |000⟩ $\oplus$ |011⟩ ) = ( |111⟩, |011⟩ ).

It has the following circuit representation:

![Figure oracle 111](img/oracle111.png)

Take the output of the circuit and apply the same circuit again,


U ( |111⟩, |011⟩ ) = ( |111⟩, |011⟩ $\oplus$ f( |111⟩ ) ) = ( |111⟩, |011⟩ $\oplus$ |011⟩ ) = ( |111⟩, |000⟩ ),

with this circuit representation:

![Figure oracle 111 again](img/oracle111again.png)

The original inputs are again recovered. And we can distinguish input results between f(3) and f(7).


In fact, the circuit is designed in such a way that U is not only unitary but also its own inverse.

U = U$^{−1}$.

Applying the circuit back to back:


U( U( |x⟩, |y⟩ ) ) = U( ( |x⟩, |y $\oplus$ f(x)⟩ ) )


= ( |x⟩, | (y $\oplus$ f(x) ) $\oplus$ f(x) ⟩ )


= ( |x⟩, | y $\oplus$ ( f(x) $\oplus$ f(x) ) ⟩


= ( |x⟩, | y $\oplus$ 0 ⟩ ) = ( |x⟩, |y⟩ )

![Figure oracles unitary](img/oracles_unitary.png)

This proof is self-consistent. We can now create quantum oracles that are implemented using unitary
gates that are self-inverse. We will be using these quantum oracles in many algorithms like Deutsch,
Deutsch-Jozsa, Grover's algorithms, etc.

### Q# exercise: Oracles
 <a id='#qexercise'></a>
1. Go to the Quantum Katas: https://github.com/Microsoft/QuantumKatas.
2. In Visual Studio (Code) open folder "DeutschJoszaAlgorithm" and Task.qs. Or use the Jupyter
    Notebook. Part 1 tasks focus on familiarize ourselves with oracles.
